In [1]:
import cv2
import numpy as np
import os

In [2]:
net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')

In [3]:
classes=[]
with open('../YOLO/coco.names','r') as f:
    classes=f.read().splitlines()

In [4]:
img = cv2.imread('../YOLO/image.jpeg')

In [5]:
height,width,_= img.shape

In [6]:
blob = cv2.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True,crop=True)
net.setInput(blob)
output_layers_names=net.getUnconnectedOutLayersNames()
layerOutputs=net.forward(output_layers_names)

In [7]:
boxes = []
confidences = []
class_ids = []

for output in layerOutputs:
    for detection in output:
        score = detection[5:]
        class_id = np.argmax(score)
        confidence = score[class_id]
        if confidence > .5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)

print(len(boxes))
indexes = cv2.dnn.NMSBoxes(boxes,confidences,.5,.4)
print(indexes.flatten())

22
[ 2 21 15 17  8 10]


In [8]:
font = cv2.FONT_HERSHEY_PLAIN
colors= np.random.uniform(0,255,size=(len(boxes)))

for i in indexes.flatten():
    x,y,w,h = boxes[i]
    label=str(classes[class_ids[i]])
    confidence=str(round(confidences[i],2))
    color=colors[i]
    cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
    cv2.putText(img,label+" "+confidence,(x,y+20),font,2,(255,255,255),2)
    

In [9]:
# for b in blob:
#     for n,img_blob in enumerate(b):
#         cv2.imshow(str(n),img_blob)

In [10]:
cv2.imshow('Image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()